# Import

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import time

# pour traitement du language (NLP)
import string
import nltk
nltk.download('wordnet')
from nltk import tokenize, pos_tag
from nltk.corpus import stopwords, wordnet
from nltk.stem import PorterStemmer, WordNetLemmatizer
from wordcloud import WordCloud
from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from gensim.corpora import Dictionary
from sklearn.manifold import TSNE

import mlflow

from sklearn import metrics as metrics_sklearn

[nltk_data] Downloading package wordnet to /home/slerendu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# GPU

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

2023-09-06 11:54:33.846863: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-06 11:54:34.167306: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-06 11:54:35.023474: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8089494032435413241
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4370071552
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14858978808541407720
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1660 SUPER, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


2023-09-06 11:54:35.950376: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-06 11:54:35.966001: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-06 11:54:35.966373: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [ ]:
import tensorflow as tf
from tensorflow import keras
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-09-06 11:54:36.060254: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-06 11:54:36.060398: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-06 11:54:36.060463: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [ ]:
print('GPU name: ', tf.config.experimental.list_physical_devices('GPU'))

GPU name:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
# Ref: https://www.tensorflow.org/guide/gpu
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

# Méthodes

### Méthodes pour les datas

In [ ]:
# pour les pie charts, pour afficher les pourcentages et les libellés uniquement avec le taux > limit_perc%
limit_perc = 5
def show_perc(pct):
    """ Affiche le pourcentage de chaque portion si la condition est vérifiée """
    return ('%.1f%%' % pct) if pct > limit_perc else ''

def show_labels(data):
    list = []
    for val, cnt in data.items():
        if (cnt*100/np.sum(data)) > limit_perc :
            list.append(val)
        else:
            list.append('')
    return list

In [ ]:
def show_unique_values(df, col, display_pie=True, silent=False):
    """
    Affiche la liste des valeurs uniques contenues dans une colonne
    Affiche le pie chart de ces données également
    """
    temp_val_count = df[col].value_counts(normalize=True)
    df_val_count = temp_val_count.rename_axis("label").to_frame("%")
    df_val_count["%"] = round(df_val_count["%"]*100, 2)
    df_val_count["%_cumul"] = df_val_count["%"].cumsum()
    # afficher la majeur partie des catégories
    sub_display = df_val_count.loc[df_val_count["%_cumul"] <= 85, :]
    
    if silent:
        return sub_display
    
    if len(sub_display) < 5:
        display(df_val_count.head())
    else:
        display(sub_display.head(len(sub_display)))
    if display_pie:
        temp_val_count.plot(kind='pie', autopct=show_perc, labels=show_labels(temp_val_count), label='')
        plt.axis('equal')
        plt.show()
    return sub_display

### Méthodes pour preprocessing

In [ ]:
def get_wordnet_pos(word):
    """Return the corresponding character for a word use in the lemmatization
    
    Parameters:
    word (str): a word
    
    Returns:
    str: the corresponding character
    """
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [ ]:
def detailled_lemmatizer(sentence):
    """Lemmatize a sentence and return it
    
    Parameters:
    sentence (list(str)): a list of words
    
    Returns:
    (list(str)): a list of lemmatized words
    """
    lemmatizer = WordNetLemmatizer()
    result = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in sentence]
    return result

In [ ]:
def clean_text(X):
    X = X.split()
    X_wo_arobas = [x for x in X if not x.startswith("@")]
    X_new = [x for x in X_wo_arobas if not x.startswith("http")]
    return ' '.join(X_new)

In [ ]:
def filter_pos_tag(df, col, tags_to_remove=[], debug=False):
    """Filter words from a list of Part-Of-Speech tags
    
    Parameters:
    df (dataframe): input dataframe
    col (str): column to process
    tags_to_remove list(str): a list of POS tag
    debug (bool): show debug elements
    
    Returns:
    list(list(str)): return a list of document. Each document is a list of words    
    """
    result = []
    if not tags_to_remove:
        print("Aucun filtre n'a été défini.")
        return df[col]
    for index, row in df.iterrows():
        temp_res = pos_tag(row[col])
        temp_res = [x for x in temp_res if x[1] not in tags_to_remove]
        # if debug:
        #     print(temp_res[0])
        result.append([x[0] for x in temp_res])
    return result

In [ ]:
def filter_exclude_words(df, col, words_to_exclude=[], debug=False):
    """Filter words from a list of specific words
    
    Parameters:
    df (dataframe): input dataframe
    col (str): column to process
    words_to_exclude list(str): a list of words
    debug (bool): show debug elements
    
    Returns:
    list(list(str)): return a list of document. Each document is a list of words    
    """
    result = []
    if not words_to_exclude:
        print("Aucun mots à exclure.")
        return df[col]
    for index, row in df.iterrows():
        result.append([x for x in row[col] if x not in words_to_exclude])
    return result

In [ ]:
def display_wordcloud(corpus, debug=False):
    """Display a WordCloud picture from a corpus
    
    Parameters:
    corpus (dict): a Counter dictionary with the frequency of each words
    debug (bool): show debug elements
    
    Returns:
    
    """
    wordcloud = WordCloud(
            random_state = 42,
            normalize_plurals = False,
            width = 600, 
            height= 300,
            max_words = 100,
            stopwords = [],
            colormap="BrBG")

    wordcloud.generate_from_frequencies(corpus)
    
    fig, ax = plt.subplots(1,1, figsize = (12,8))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")

In [ ]:
def clean_textism(sentence):
    neo_sentence = []
    for word in sentence:
        if word == 'u':
            neo_sentence.append('you')
        elif word == 'r':
            neo_sentence.append('are')
        elif word == 'ur':
            neo_sentence.append('your')
        elif word == 'some1':
            neo_sentence.append('someone')
        elif word == 'yrs':
            neo_sentence.append('years')
        elif word == 'hrs':
            neo_sentence.append('hours')
        elif word == 'mins':
            neo_sentence.append('minutes')
        elif word == 'secs':
            neo_sentence.append('seconds')
        elif word == 'pls' or word == 'plz':
            neo_sentence.append('please')
        elif word == '2morow':
            neo_sentence.append('tomorrow')
        elif word == '2day':
            neo_sentence.append('today')
        elif word == '2nite':
            neo_sentence.append('tonight')
        elif word == '4got' or word == '4gotten':
            neo_sentence.append('forget')
        elif word == 'amp' or word == 'quot' or word == 'lt' or word == 'gt' or word == '½25':
            neo_sentence.append('')
        else:
            neo_sentence.append(word)
    return neo_sentence

In [ ]:
def preprocessing_words(df, col, stem_or_lemma="stem", debug=False):
    """Compute all the basics string tranformations in NLP
    Normalisation, Tokenization, Remove of stopwords, Stemmation or Lemmatization
    
    Parameters:
    df (dataframe): input dataframe
    col (str): column to process
    stem_or_lemma (str): choose between stemmation or lemmatization
    debug (bool): show debug elements
    
    Returns:
    list(list(str)): return a list of document. Each document is a list of words
    """
    result = []    
    # building stopwords list
    stopW = stopwords.words('english')
    stopW.extend(string.punctuation)
    
    for index, row in df.iterrows():        
        temp_res = ""
        # normalisation
        temp_res = row[col].lower()
        # tokenization
        tk = tokenize.TweetTokenizer(reduce_len=True)
        temp_res = tk.tokenize(temp_res)
        
        # clean the sms language to usefull langage
        temp_res = clean_textism(temp_res)
        
        # remove stopwords
        temp_res = [word for word in temp_res if word not in stopW]
        
        # stemmation or lemmatization
        if stem_or_lemma == "stem":
            stemmer = PorterStemmer()
            temp_res = [stemmer.stem(elt) for elt in temp_res]
        elif stem_or_lemma == "lemma":
            temp_res = detailled_lemmatizer(temp_res)
        else:
            print("stem or lemma only")
            
        result.append(temp_res)
    return result

In [ ]:
def natural_language_processing(
    df, col, stem_or_lemma="stem", tags_to_remove=[], words_to_exclude=[],
    max_threshold=999, min_threshold=0, word_cloud=True, debug=False):
    """Compute NLP preprocessing methods
    
    Parameters:
    df (dataframe): input dataframe
    col (str): column to process
    stem_or_lemma (str): choose between stemmation or lemmatization
    tags_to_remove list(str): a list of POS tag
    words_to_exclude list(str): a list of words
    max_threshold (int): upper threshold to filter word frequency
    min_threshold (int): lower threshold to filter word frequency
    word_cloud (bool): display WordCloud representation
    debug (bool): show debug elements
    
    Returns:
    list(list(str)): return a list of document. Each document is a list of words    
    """
    # df = input_df.copy()
    # supprimer des mots spécifiques à twitter (commençant par @ et hhtp)
    df[col] = df[col].apply(clean_text)
    
    # preprocessing part
    preproc_res = preprocessing_words(df, col, stem_or_lemma=stem_or_lemma, debug=debug)
    df.insert(0, 'preproc_text', preproc_res)
    
    # filter by pos tag part
    filtpos_res = filter_pos_tag(df, "preproc_text", tags_to_remove=tags_to_remove, debug=debug)
    df.insert(0, 'filtpos_text', filtpos_res)
    
    # filter by excluding words
    filtexcl_words = filter_exclude_words(df, "filtpos_text", words_to_exclude=words_to_exclude, debug=debug)
    df.insert(0, 'filtexcl_words', filtexcl_words)
    
    if debug:
        display(df[[col, 'preproc_text', 'filtpos_text', 'filtexcl_words']].head())
    
    # concatenate all documents into a single corpus
    corpus = df["filtexcl_words"].tolist()
    corpus = [item for sublist in corpus for item in sublist]
    
    word_counts = Counter(corpus)
    print("Il y a un total de", len(word_counts), "mots différents dans tout le corpus.")
    word_counts_threshold = {x: count for x, count in word_counts.items() if count > min_threshold}
    word_counts_threshold = {x: count for x, count in word_counts_threshold.items() if count < max_threshold}
    print("Après filtrage, on garde les mots aparaissant plus de", min_threshold,
          "fois et moins de", max_threshold, "fois. Il reste alors",
          len(word_counts_threshold), "mots différents dans tout le corpus.\n")
    word_counts = Counter(word_counts_threshold)
    
    if debug:
        print("------------ Top 20 plus communs : ------------")
        display_most_common = [(i, word_counts[i], word_counts[i] / len(corpus) * 100.0) for i, count in word_counts.most_common(20)]
        for elt in display_most_common:
            print(elt)
        print("\n------------ Top 20 moins communs : ------------")
        display_least_common = [(i, word_counts[i], word_counts[i] / len(corpus) * 100.0) for i, count in word_counts.most_common()[-20:]]
        for elt in display_least_common:
            print(elt)
        
    simplified_corpus = word_counts_threshold.keys()
    
    # display wordcloud part
    if word_cloud:
        display_wordcloud(word_counts_threshold, debug=debug)
    
    return simplified_corpus

### Méthodes pour BOW and Word Embedding

In [ ]:
def representation_by_tf_idf(corpus):
    """Compute corpus into a tf-idf vectorisation
    
    Parameters:
    corpus (list(list(str)): a list of documents
    
    Returns:
    a matrix of TF-IDF features
    the list of features names
    """
    vect = TfidfVectorizer(ngram_range=(1, 1))
    tfidf_mat = vect.fit_transform(corpus)
    features_names = vect.get_feature_names_out()
    return tfidf_mat, features_names

In [ ]:
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
def representation_by_doc2vec(corpus):
    X = [TaggedDocument(doc, [i]) for i, doc in enumerate(corpus)]

    # Train doc2vec model
    doc2vec = Doc2Vec()
    doc2vec.build_vocab(X)
    doc2vec.train(X, total_examples=doc2vec.corpus_count, epochs=doc2vec.epochs)

    # Vectorize text
    X = [doc2vec.infer_vector(doc.words) for doc in X]
    return X

### Méthodes pour les métriques

In [ ]:
def display_confusion_matrix(actual_class, predict_class, display_labels=None, specifity=False):
    """display the confusion matrix
    
    Parameters:
    actual_class : serie of the actual classes
    predict_class : serie of the predicted classes
    display_labels list(str)) : list of labels to show in the confusion matrix
    
    Returns:
    
    """
    print("Calcul et affichage de la matrice de confusion")
    # pour calculer la matrice de confusion
    cm = metrics_sklearn.confusion_matrix(actual_class, predict_class)#, labels=display_labels)
    cm = cm.T
    
    # pour afficher la matrice de confusion
    displ = metrics_sklearn.ConfusionMatrixDisplay(cm, display_labels=display_labels)
    displ.plot()
    displ.ax_.xaxis.tick_top()
    displ.ax_.xaxis.set_label_position('top') 
    plt.xlabel('Actual Label')
    plt.ylabel('Predicted Label')
    plt.gcf().axes[0].tick_params()
    plt.gcf().axes[1].tick_params()
    plt.show()
    
    # compute specificity
    if specifity:
        speci = cm[0, 0]/(cm[0, 0] + cm[1, 0])
        print("Spécificité :", speci)
        return speci
    
    return False

In [ ]:
def plot_roc_curve(model_name, y_test, y_test_pred):  
    print("Calcul et affichage de la courbe ROC")
    [fpr_te, tpr_te, thr_te] = metrics_sklearn.roc_curve(y_test, y_test_pred)#, pos_label=1)
    plt.plot(fpr_te, tpr_te, color='coral', lw=2, label="")
    
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('1 - Specificity', fontsize=14)
    plt.ylabel('Sensitivity', fontsize=14)
    plt.title("ROC du classifier {}".format(model_name))

In [ ]:
def display_classification_metrics(estimator, y_test, y_test_pred, list_metrics=["accuracy"], all_metrics=False, roc_curve=False, beta=1):
    print("Calcul et affichage des différentes métriques")
    result = {}
    if all_metrics:
        list_metrics = ["accuracy", "recall", "precision", "auc", "f1", "fbeta"]
    for met in list_metrics:
        if met == "accuracy":
            te_res = metrics_sklearn.accuracy_score(y_test, y_test_pred).round(3)
            print("Accuracy :", te_res)
        elif met == "precision":     
            te_res = metrics_sklearn.precision_score(y_test, y_test_pred, average='macro').round(3)
            print("Précision :", te_res)
        elif met == "recall":  
            te_res = metrics_sklearn.recall_score(y_test, y_test_pred, average='macro').round(3)
            print("Recall :", te_res)
        elif met == "auc":    
            te_res = metrics_sklearn.roc_auc_score(y_test, y_test_pred, average='macro').round(3)
            print("AUC :", te_res)
            if roc_curve:
                plot_roc_curve(type(estimator).__name__, y_test, y_test_pred)
        elif met == "f1":   
            te_res = metrics_sklearn.f1_score(y_test, y_test_pred, average='macro').round(3)
            print("F1-score :", te_res)
        elif met == "fbeta":
            te_res = metrics_sklearn.fbeta_score(y_test, y_test_pred, beta=beta, average='macro').round(3)
            print("Fbeta-score (beta=", beta,  ") :", te_res)
        else:
            print("La métrique", met, "n'est pas dans la liste suivante accuracy, recall, precision, auc, f1 et fbeta. Veuillez vérifier la saisie.")
        result[met] = te_res
    return result

### Méthodes pour les modèles de régression

In [ ]:
from sklearn import naive_bayes
def get_naive_bayes_model(model_name="gaussian"):
    """return an instance of the model selected
    
    Parameters:
    model_name (str) : names of naive bayes model used
    
    Returns:
    instance of the model
    """
    if model_name == "gaussian":
        return naive_bayes.GaussianNB()
    elif model_name == "multinomial":
        return naive_bayes.MultinomialNB()
    elif model_name == "bernoulli":
        return naive_bayes.BernoulliNB()
    else:
        return -1

### Méthodes pour les Réseaux de neurones

In [ ]:
from keras.preprocessing import text
def tokenize_corpus(corpus):
    tokenizer = text.Tokenizer()
    tokenizer.fit_on_texts(corpus)
    return tokenizer

In [ ]:
from gensim import models
def word_embedding_by_word2vec(embed_dim, data):
    w2v_model = models.word2vec.Word2Vec(data, vector_size=embed_dim, workers=8)
    # w2v_model.build_vocab(data)
    w2v_model.train(data, total_examples=len(data), epochs=50)
    return w2v_model

In [ ]:
# !wget http://nlp.stanford.edu/data/glove.twitter.27B.zip
# !unzip data/glove.twitter.27B.zip
def word_embedding_by_glove(corpus):
    GLOVE_EMB = 'glove.twitter.27B.200d.txt'
      
    # compute embeddings index from GloVe
    embeddings_index = {}
    f = open(GLOVE_EMB)
    for line in f:
        values = line.split()
        word = value = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('Found %s word vectors.' %len(embeddings_index))
    return embeddings_index   

In [ ]:
def compute_corpus_to_word_embedding(corpus, model_name="word2vec"):
    tokenizer = tokenize_corpus(corpus)
    word_index = tokenizer.word_index
    vocab_size = len(tokenizer.word_index) + 1  
    
    if model_name == "word2vec":
        embed_dim = 800
        
        # word embedding using Word2Vec
        w2v_model = word_embedding_by_word2vec(embed_dim, corpus)
        word_index = w2v_model.wv.key_to_index
        # vocab_size = len(word_index)
        
        # compute embedding matrix
        embedding_matrix = np.zeros((vocab_size, embed_dim))
        for word, i in tokenizer.word_index.items():
            if w2v_model.wv.__contains__(word):
                embedding_matrix[i] = w2v_model.wv.__getitem__(word)
    elif model_name == "glove":
        embed_dim = 200
        
        # word embedding using GloVe
        embeddings_index = word_embedding_by_glove(corpus)
        
        # compute embedding matrix
        embedding_matrix = np.zeros((vocab_size, embed_dim))
        for word, i in word_index.items():
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
                
    print("Embedding Matrix Shape:", embedding_matrix.shape)
    return embedding_matrix, tokenizer, embed_dim

In [ ]:
import keras.backend as K
def specificity(y_true, y_pred):
    """
    param:
    y_pred - Predicted labels
    y_true - True labels 
    Returns:
    Specificity score
    """
    neg_y_true = 1 - y_true
    neg_y_pred = 1 - y_pred
    fp = K.sum(neg_y_true * y_pred)
    tn = K.sum(neg_y_true * neg_y_pred)
    specificity = tn / (tn + fp + K.epsilon())
    return specificity

In [ ]:
def display_result_plot(history):
    spec,  val_spec  = history.history['specificity'], history.history['val_specificity']
    loss, val_loss = history.history['loss'], history.history['val_loss']
    epochs = range(len(spec))

    plt.plot(epochs, spec, 'b', label='Training spec')
    plt.plot(epochs, val_spec, 'r', label='Validation spec')
    plt.title('Training and validation specificity')
    plt.legend()

    plt.figure()

    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()

In [ ]:
def create_neural_network_model(embedding_layer):
    # instance de neural network model
    model = keras.Sequential()
    
    # on ajoute toutes les couches nécessaires
    model.add(embedding_layer)
    model.add(layers.Conv1D(128, 5, activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(16, activation="relu"))
    model.add(layers.Dense(16, activation="relu"))
    model.add(layers.Dense(1, activation="sigmoid"))
    
    # on crée les fonctions de loss et d'optimisation
    loss = losses.BinaryCrossentropy(from_logits=True)
    optimizer = optimizers.Adam(learning_rate=2e-4, epsilon=1e-08)
    
    # on compile le modèle
    model.compile(loss=loss, optimizer=optimizer, metrics=[specificity])
    model.summary()
    
    return model

In [ ]:
def train_neural_network_model(model, X_train, y_train, X_test, y_test):
    # on crée les méthodes de callback permettant d'arrêter l'entraînement dès que certaines conditions sont remplies
    callbacks = [
        ReduceLROnPlateau(monitor='val_loss', patience=10, cooldown=0),
        EarlyStopping(monitor='val_specificity', min_delta=1e-4, patience=10)
    ]
 
    history = model.fit(X_train, y_train, batch_size=1024, epochs=20,
                        validation_data=(X_test, y_test), callbacks=callbacks,
                        verbose=1)
    
    return history

# Preparing dataset

In [ ]:
data = pd.read_csv("data/training.1600000.processed.noemoticon.csv", sep=',',
                   encoding="ISO-8859-1", names=["target", "id", "date", "flag", "user", "text"])

In [ ]:
data.head()

In [ ]:
data["target"].replace(4, 1, inplace=True)

In [ ]:
show_unique_values(data, "target")

Sentiment du tweet 0 pour négatif et 4 pour positif  
Dans tous les cas, nous n'avons besoin que du texte du tweet et de la target

In [ ]:
data.drop(columns=["id", "date", "flag", "user"], inplace=True)

In [ ]:
print("The dataset is containing", data.shape[0], "lines and", data.shape[1], "columns.")

On commence par prendre un échantillon des données dans un premier temps : 10 000 de chaque target.

In [ ]:
data_words = data.groupby(['target']).apply(pd.DataFrame.sample, n=10000, replace=True, random_state=42).reset_index(drop=True)

In [ ]:
print("The sample dataset is containing", data_words.shape[0], "lines and", data_words.shape[1], "columns.")

# Preprocessing

In [ ]:
words_to_exclude = [
    "...", ".."
]
tags_to_remove=["NNP", "NN", "VBG", "VBN", "CD"]

corpus = natural_language_processing(
    data_words, "text", stem_or_lemma="lemma", debug=False, word_cloud=True,
    tags_to_remove=tags_to_remove, words_to_exclude=words_to_exclude)

In [ ]:
data_words.head()

In [ ]:
# drop des colonnes inutiles
data_words.drop(columns=["filtpos_text", "preproc_text", "text"], inplace=True)

In [ ]:
def join_back(text):
    return ' '.join(text)

data_words['sentence'] = data_words['filtexcl_words'].apply(join_back)
data_words.rename(columns={"filtexcl_words": "list_words"}, inplace=True)

In [ ]:
data_words.head()

# Approche modèles sur mesure simple

Utilisation du bag of words pour le test des modèles sur mesure simple

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=800)# vectorizing words and storing in variable X(predictor)
X = cv.fit_transform(data_words["sentence"]).toarray()
print(X.shape)
y = data_words.target.values# y size
print(y.shape)

In [ ]:
# 3 parties : train 60%, validation 20% and test 20%
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(
    X,
    y,
    test_size=0.4,
    stratify=y,
    random_state=42,
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp,
    y_temp,
    test_size=0.5,
    stratify=y_temp,
    random_state=42,
)

### Test des 3 Naive Bayes models

In [ ]:
for nb_model in ["gaussian", "multinomial", "bernoulli"]:
    model = get_naive_bayes_model(nb_model)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    res = display_classification_metrics(model, y_test, y_pred, list_metrics=["precision", "auc"], roc_curve=False)

Bernoulli est meilleur comme model de référence

### Test avec Naive Bayes Bernoulli

In [ ]:
model = get_naive_bayes_model("bernoulli")
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
res = display_classification_metrics(model, X_val, y_val, y_pred, list_metrics=["precision", "auc"], roc_curve=True)

In [ ]:
speci_bayes = display_confusion_matrix(actual_class=y_val, predict_class=y_pred, display_labels=["Negative", "Positive"], specifity=True)

In [ ]:
truc = bidule

### Test avec XGBoost

In [ ]:
from sklearn import ensemble

In [ ]:
model_xgb = ensemble.GradientBoostingClassifier(random_state=42)
# model_xgb = ensemble.GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=42)
model_xgb.fit(X_train, y_train)
y_pred_xgb = model_xgb.predict(X_val)
res_xgb = display_classification_metrics(model_xgb, y_val, y_pred, list_metrics=["precision", "auc"], roc_curve=True)

In [ ]:
speci_xgboost = display_confusion_matrix(actual_class=y_val, predict_class=y_pred_xgb, display_labels=["Negative", "Positive"], specifity=True)

### XGBoost GridSearchCV

In [ ]:
start = time.time()
model_xgb = ensemble.GradientBoostingClassifier(random_state=42)

In [ ]:
# learning_rate = 0.1
# n_estimators = 100
# max_depth = 3

# 16 minutes avec ces paramètres
parameters = {
    "learning_rate": [0.05, 0.1, 0.15],
    "max_depth":[1, 3, 5],
    "n_estimators":[50, 100, 150]
    }

In [ ]:
from sklearn.metrics import make_scorer
scorer = make_scorer(specificity)

In [ ]:
from sklearn import model_selection
grid_search_xgb = model_selection.GridSearchCV(
    estimator=model_xgb,
    param_grid=parameters,
    # scoring = "accuracy",
    scoring = scorer,
    n_jobs = None,
    cv = 5
)

In [ ]:
xgb_inst=grid_search_xgb.fit(X_train, y_train)

In [ ]:
y_pred_xgb =xgb_inst.predict(X_test)

In [ ]:
end = time.time()
delta = round(end - start, 0)
print("Execution time is:", delta, "s, soit", round(delta/60, 0), "min, soit" , round(delta/3600, 0), "h.")

In [ ]:
print("Les meilleures paramètres sont les suivants :\n", grid_search_xgb.best_params_ )

In [ ]:
model_xgb = ensemble.GradientBoostingClassifier(**grid_search_xgb.best_params_)
model_xgb.fit(X_train, y_train)
y_pred_xgb = model_xgb.predict(X_val)
res_xgb = display_classification_metrics(model_xgb, y_val, y_pred, list_metrics=["precision", "auc"], roc_curve=True)

In [ ]:
speci_xgboost = display_confusion_matrix(actual_class=y_val, predict_class=y_pred_xgb, display_labels=["Negative", "Positive"], specifity=True)

In [ ]:
res = display_classification_metrics(model_xgb, y_val, y_pred_xgb, all_metrics=True, roc_curve=True, beta=1)

In [ ]:
from sklearn import metrics
def display_classification_report(y_true, y_pred, display_labels=None):
    print("Calcul et affichage du rapport de classification")
    clf_report = metrics.classification_report(y_true, y_pred, target_names=display_labels)
    print(clf_report)

In [ ]:
display_classification_report(y_val, y_pred_xgb, display_labels=["Negative", "Positive"])

In [ ]:
truc = bidule

# ML

### Splitting data

In [ ]:
from sklearn import model_selection

In [ ]:
# 3 parties : train 60%, validation 20% and test 20%
X, y = np.array(data_words['sentence']), np.array(data_words['target'])

X_train, X_temp, y_train, y_temp = model_selection.train_test_split(
    X,
    y,
    test_size=0.4,
    stratify=y,
    random_state=42,
)

X_val, X_test, y_val, y_test = model_selection.train_test_split(
    X_temp,
    y_temp,
    test_size=0.5,
    stratify=y_temp,
    random_state=42,
)

In [ ]:
print("Train data shape :", X_train.shape)
print("Test data shape :", X_test.shape)
print("Validation data shape :", X_val.shape)

### Word Embedding

In [ ]:
# corpus = list of documents
corpus = data_words["list_words"].tolist()

In [ ]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import layers, losses, optimizers
from tensorflow import keras
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [ ]:
MAX_SENTENCE_LENGTH = 280 # max characters in a tweet

In [ ]:
tokenizer = tokenize_corpus(corpus)
    
X_train = sequence.pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=MAX_SENTENCE_LENGTH)
X_test  = sequence.pad_sequences(tokenizer.texts_to_sequences(X_test) , maxlen=MAX_SENTENCE_LENGTH)
X_val  = sequence.pad_sequences(tokenizer.texts_to_sequences(X_val) , maxlen=MAX_SENTENCE_LENGTH)

On teste les word embedding Word2Vec et GloVe. Chacun a des paramètres différents.

### Simple neural network

In [ ]:
# speci_res = {}

# for word_embed_model in ["word2vec", "glove"]:
#     print("-"*70, "Embedding model", word_embed_model, "-"*70)
    
#     # word embedding
#     embed_mat, tokenizer, embed_dim = compute_corpus_to_word_embedding(corpus, model_name=word_embed_model)
#     vocab_size = len(tokenizer.word_index) + 1
    
#     # create embedding layer
#     embedding_layer = layers.Embedding(
#         input_dim=vocab_size, output_dim=embed_dim, weights=[embed_mat],
#         input_length=MAX_SENTENCE_LENGTH, trainable=False)
    
#     # create the neural network
#     model = create_neural_network_model(embedding_layer)
    
#     # train the neural network
#     history = train_neural_network_model(model, X_train, y_train, X_test, y_test)
    
#     # show the result into a graph
#     display_result_plot(history)
    
#     # compute confusion matrix and specificity
#     y_pred = model.predict(X_val)
#     y_pred = np.where(y_pred>=0.5, 1, 0)
#     speci = display_confusion_matrix(actual_class=y_val, predict_class=y_pred, display_labels=["Negative", "Positive"], specifity=True)
#     speci_res[word_embed_model] = speci

In [ ]:
# speci_res

In [ ]:
from sklearn import model_selection
from scikeras.wrappers import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model():
    # word embedding
    embed_mat, tokenizer, embed_dim = compute_corpus_to_word_embedding(corpus, model_name="glove")
    vocab_size = len(tokenizer.word_index) + 1

    # create embedding layer
    embedding_layer = layers.Embedding(
    input_dim=vocab_size, output_dim=embed_dim, weights=[embed_mat],
    input_length=MAX_SENTENCE_LENGTH, trainable=False)
    
    # create model
    model = keras.Sequential()
    model.add(embedding_layer)
    model.add(layers.Conv1D(128, 5, activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(16, activation="relu"))
    model.add(layers.Dense(16, activation="relu"))
    model.add(layers.Dense(1, activation="sigmoid"))
    
    # on crée les fonctions de loss et d'optimisation
    loss = losses.BinaryCrossentropy(from_logits=True)
    optimizer = optimizers.Adam(learning_rate=2e-4, epsilon=1e-08)
    
    # on compile le modèle
    model.compile(loss=loss, optimizer=optimizer, metrics=[specificity])
    return model

In [ ]:
start = time.time()
# create model
model = KerasClassifier(model=create_model, verbose=0)
# define the grid search parameters
batch_size = [50, 100]
epochs = [10, 20]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = model_selection.GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=None, cv=3, error_score='raise')
grid_result = grid.fit(X_train, y_train)

In [ ]:
end = time.time()
delta = round(end - start, 0)
print("Execution time is:", delta, "s, soit", round(delta/60, 0), "min, soit" , round(delta/3600, 0), "h.")

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
truc = bidule

## LSTM

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D, Dense, LSTM, Conv1D, Embedding

In [ ]:
def create_lstm_neural_network_model(embedding_layer):
    # on crée la séquence avec toutes les couches de notre réseau de neurones
    model = keras.Sequential([
        embedding_layer,
        Bidirectional(LSTM(100, dropout=0.3, return_sequences=True)),
        Bidirectional(LSTM(100, dropout=0.3, return_sequences=True)),
        Conv1D(100, 5, activation='relu'),
        GlobalMaxPool1D(),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid'),
    ],
    name="Sentiment_Model")
    
    # on crée les fonctions de loss et d'optimisation
    loss = losses.BinaryCrossentropy(from_logits=True)
    optimizer = optimizers.Adam(learning_rate=2e-4, epsilon=1e-08)
    
    # on compile le modèle
    model.compile(loss=loss, optimizer=optimizer, metrics=[specificity])
    model.summary()
    
    return model

In [ ]:
speci_res = {}

for word_embed_model in ["word2vec", "glove"]:
    print("-"*70, "Embedding model", word_embed_model, "-"*70)
    
    # word embedding
    embed_mat, tokenizer, embed_dim = compute_corpus_to_word_embedding(corpus, model_name=word_embed_model)
    vocab_size = len(tokenizer.word_index) + 1
    
    # create embedding layer
    embedding_layer = layers.Embedding(
        input_dim=vocab_size, output_dim=embed_dim, weights=[embed_mat],
        input_length=MAX_SENTENCE_LENGTH, trainable=False)
    
    # create the neural network
    model = create_lstm_neural_network_model(embedding_layer)
    
    # train the neural network
    history = train_neural_network_model(model, X_train, y_train, X_test, y_test)
    
    # show the result into a graph
    display_result_plot(history)
    
    # compute confusion matrix and specificity
    y_pred = model.predict(X_val)
    y_pred = np.where(y_pred>=0.5, 1, 0)
    speci = display_confusion_matrix(actual_class=y_val, predict_class=y_pred, display_labels=["Negative", "Positive"], specifity=True)
    speci_res[word_embed_model] = speci

In [ ]:
speci_res

## GRU

In [ ]:
from tensorflow.keras.layers import GRU

In [ ]:
def create_gru_neural_network_model(embedding_layer):
    # instance de neural network model
    model = keras.Sequential()
    
    # on ajoute toutes les couches nécessaires
    model.add(embedding_layer)
    model.add(layers.Conv1D(128, 5, activation='relu'))
    # model.add(layers.Flatten())
    model.add(layers.GRU(64))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(16, activation="relu"))
    model.add(layers.Dense(1, activation="sigmoid"))
    
    # on crée les fonctions de loss et d'optimisation
    loss = losses.BinaryCrossentropy(from_logits=False)
    optimizer = optimizers.Adam(learning_rate=2e-4, epsilon=1e-08)
    
    # on compile le modèle
    model.compile(loss=loss, optimizer=optimizer, metrics=[specificity])
    model.summary()
    
    return model

In [ ]:
speci_res = {}

for word_embed_model in ["word2vec", "glove"]:
    print("-"*70, "Embedding model", word_embed_model, "-"*70)
    
    # word embedding
    embed_mat, tokenizer, embed_dim = compute_corpus_to_word_embedding(corpus, model_name=word_embed_model)
    vocab_size = len(tokenizer.word_index) + 1
    
    # create embedding layer
    embedding_layer = layers.Embedding(
        input_dim=vocab_size, output_dim=embed_dim, weights=[embed_mat],
        input_length=MAX_SENTENCE_LENGTH, trainable=False)
    
    # create the neural network
    model = create_gru_neural_network_model(embedding_layer)
    
    # train the neural network
    history = train_neural_network_model(model, X_train, y_train, X_test, y_test)
    
    # show the result into a graph
    display_result_plot(history)
    
    # compute confusion matrix and specificity
    y_pred = model.predict(X_val)
    y_pred = np.where(y_pred>=0.5, 1, 0)
    speci = display_confusion_matrix(actual_class=y_val, predict_class=y_pred, display_labels=["Negative", "Positive"], specifity=True)
    speci_res[word_embed_model] = speci

In [ ]:
speci_res

# BERT

In [ ]:
from transformers import *
from transformers import BertTokenizer, TFBertModel, BertConfig, TFBertForSequenceClassification

In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

In [ ]:
# for sent in data_words["sentence"]:
#     tokens=bert_tokenizer.tokenize(sent)

In [ ]:
input_ids=[]
attention_masks=[]

for sent in data_words["sentence"]:
    bert_inp = bert_tokenizer.encode_plus(
        sent, add_special_tokens=True, max_length=64, pad_to_max_length=True,
        return_attention_mask=True)
    input_ids.append(bert_inp['input_ids'])
    attention_masks.append(bert_inp['attention_mask'])

input_ids=np.asarray(input_ids)
attention_masks=np.array(attention_masks)
labels=np.array(data_words['target'])

In [ ]:
len(input_ids), len(attention_masks), len(labels)

In [ ]:
bert_model.summary()

In [ ]:
# callbacks = [tf.keras.callbacks.ModelCheckpoint(
#     filepath=model_save_path,save_weights_only=True,monitor='val_loss',
#     mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]

In [ ]:
import tensorflow as tf

In [ ]:
loss = losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = optimizers.Adam(learning_rate=2e-5, epsilon=1e-08)
# learning_rate=2e-5 recommended by the Bert team

bert_model.compile(loss=loss,optimizer=optimizer,metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [ ]:
train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.2)

In [ ]:
history=bert_model.fit(
    [train_inp,train_mask],train_label,batch_size=32,epochs=4,
    validation_data=([val_inp,val_mask],val_label))

In [ ]:
def display_result_plot_accuracy(history):
    spec,  val_spec  = history.history['categorical_accuracy'], history.history['val_categorical_accuracy']
    loss, val_loss = history.history['loss'], history.history['val_loss']
    epochs = range(len(spec))

    plt.plot(epochs, spec, 'b', label='Training spec')
    plt.plot(epochs, val_spec, 'r', label='Validation spec')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.figure()

    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()

In [ ]:
display_result_plot_accuracy(history)

In [ ]:
# compute confusion matrix and specificity
y_pred = bert_model.predict(X_val)

In [ ]:
y_pred

In [ ]:
pred_labels = y_pred["logits"].argmax(axis=1)

In [ ]:
pred_labels

In [ ]:
pred_res = []
for tup in y_pred["logits"]:
    if tup[1]>=0.5:
        pred_res.append(1)
    else:
        pred_res.append(0)

In [ ]:
speci = display_confusion_matrix(actual_class=y_val, predict_class=pred_labels, display_labels=["Negative", "Positive"], specifity=True)